In [1]:
library(BRVM)
library(tidyverse)
library(tibble)
library(repr)
library(dplyr)
library(formattable)
library(scales)
library(readr)
library(rvest)
library(openxlsx)
library(janitor)

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


== Welcome to BRVM ================================================================
If you find this package useful, please leave a star: 
   https://github.com/Koffi-Fredysessie/BRVM

If you encounter a bug or want to request an enhancement please file an issue at:
   https://github.com/Koffi-Fredysessie/BRVM/issues

Thank you for using BRVM!


-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.0     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: 'scales'

In [2]:
# Extract BRVM Info with BRVM_ticker_desc
brvm_info <- BRVM_ticker_desc()

# Extract tickers from ticker_info
tickers <- brvm_info$Ticker

In [3]:
# Retrieve BRVM information for the BRVM tickers
brvm_info_list <- BRVM_get(.symbol = tickers, .from = "2020-01-01", .to = Sys.Date() - 1)

ABJCBICCBNBCBOABBOABFBOACBOAMBOANBOASCABCCBIBFCFACCIECECOCETITFTSCNEICNSBCNTLCONTBFORACORGTPALCPRSCSAFCSCRCSDCCSDSCSEMCSGBCSHECSIBCSICCSIVCSLBCSMBCSNTSSOGCSPHCSTACSTBCSVOCTTLCTTLSUNLCUNXC



In [4]:
# Merge the BRVM information retrieved for the specified tickers
# with additional BRVM information based on the common column "Ticker"
BRVM_frame <- merge(brvm_info_list, brvm_info, by = "Ticker")

In [7]:
# Display a message indicating the process of loading BRVM Market Cap information
cat("Loading BRVM Market Cap Info\n")

# Load BRVM market cap data
B_Market <- BRVM_cap()

# Rename the columns
B_Market <- B_Market %>%
  rename(Ticker = Symbol,
         `Market Cap` = `Global capitalization`,
         `Market Percentage` = `Global capitalization (%)`) %>%
  select(Ticker, `Market Cap`, `Market Percentage`)

# Format the "Market Cap" column as currency
B_Market$`Market Cap` <- currency(B_Market$`Market Cap`)

# Sort the data frame by "Market Cap" column in descending order
B_Market <- arrange(B_Market, desc(`Market Cap`))

# Convert Market Percentage column to numeric
B_Market <- B_Market %>%
mutate(`Market Percentage` = as.numeric(`Market Percentage`))

# Calculate percentage
B_Market$`Market Percentage` <- as.numeric(B_Market$`Market Percentage`)

# Print the modified data
print(B_Market)

Loading BRVM Market Cap Info
# A tibble: 47 x 3
   Ticker `Market Cap`      `Market Percentage`
   <chr>  <formttbl>                      <dbl>
 1 SNTS   1,789,500,000,000               22.5 
 2 ORAC   1,625,571,226,500               20.5 
 3 SGBC   549,422,202,600                  6.92
 4 ECOC   373,793,574,000                  4.71
 5 ETIT   325,513,920,996                  4.1 
 6 SIBC   283,250,000,000                  3.57
 7 CBIBF  280,000,000,000                  3.53
 8 BOABF  160,380,000,000                  2.02
 9 ORGT   152,713,068,200                  1.92
10 ONTBF  150,960,000,000                  1.9 
# i 37 more rows


In [8]:
# Merge BRVM dataframes by Ticker
BRVM_columns <- merge(BRVM_frame, B_Market, by = "Ticker") %>%

# Rename Company name column to Company
rename(Company = `Company name`) %>%

# Select relevant columns for BRVM analysis
select(Ticker,Sector, Company, Country, Date, Open, High, Low, Close, Volume, `Market Cap`, `Market Percentage`) %>%

# Arrange data by Market Cap in descending order
arrange(desc(`Market Cap`))

glimpse(BRVM_columns)

Rows: 41,665
Columns: 12
$ Ticker              <chr> "SNTS", "SNTS", "SNTS", "SNTS", "SNTS", "SNTS", "S~
$ Sector              <fct> PUBLIC SERVICE, PUBLIC SERVICE, PUBLIC SERVICE, PU~
$ Company             <chr> "SONATEL SENEGAL", "SONATEL SENEGAL", "SONATEL SEN~
$ Country             <fct> SENEGAL, SENEGAL, SENEGAL, SENEGAL, SENEGAL, SENEG~
$ Date                <date> 2020-01-02, 2020-01-03, 2020-01-06, 2020-01-07, 2~
$ Open                <dbl> 16500, 16500, 16695, 16005, 16000, 15000, 15010, 1~
$ High                <dbl> 16900, 16800, 16695, 16495, 16000, 15500, 15500, 1~
$ Low                 <dbl> 16500, 16500, 16500, 15995, 15000, 14995, 15010, 1~
$ Close               <dbl> 16900, 16800, 16500, 16000, 15000, 15500, 15500, 1~
$ Volume              <dbl> 497, 26, 222, 831, 2846, 24709, 18647, 69461, 3453~
$ `Market Cap`        <formttbl> 1,789,500,000,000, 1,789,500,000,000, 1,789,5~
$ `Market Percentage` <dbl> 22.54, 22.54, 22.54, 22.54, 22.54, 22.54, 22.54, 2~


In [9]:
# Sort dataframe in ascending order by a specific column
BRVM_columns_asc <- arrange(BRVM_columns, Date)

# Sort dataframe in descending order by a specific column
BRVM_columns_desc <- arrange(BRVM_columns, desc(Date))

In [10]:
# Sort dataframe in descending 
glimpse(BRVM_columns_desc)

Rows: 41,665
Columns: 12
$ Ticker              <chr> "SNTS", "ORAC", "SGBC", "ECOC", "ETIT", "SIBC", "C~
$ Sector              <fct> PUBLIC SERVICE, PUBLIC SERVICE, FINANCE, FINANCE, ~
$ Company             <chr> "SONATEL SENEGAL", "ORANGE COTE D'IVOIRE", "SOCIET~
$ Country             <fct> SENEGAL, IVORY COAST, IVORY COAST, IVORY COAST, TO~
$ Date                <date> 2024-03-20, 2024-03-20, 2024-03-20, 2024-03-20, 2~
$ Open                <dbl> 17850, 10800, 17650, 6800, 18, 5790, 8665, 7295, 2~
$ High                <dbl> 17885, 10800, 17750, 6800, 18, 5790, 8750, 7300, 2~
$ Low                 <dbl> 17860, 10750, 17650, 6700, 17, 5600, 8665, 7200, 2~
$ Close               <dbl> 17885, 10750, 17700, 6780, 18, 5780, 8700, 7295, 2~
$ Volume              <dbl> 1834, 1563, 469, 1342, 21960, 446, 62, 678, 252, 1~
$ `Market Cap`        <formttbl> 1,789,500,000,000, 1,625,571,226,500, 549,422~
$ `Market Percentage` <dbl> 22.54, 20.47, 6.92, 4.71, 4.10, 3.57, 3.53, 2.02, ~


In [35]:
# Select the first 40 rows of BRVM_columns_desc to create Realtime_BRVM
Realtime_BRVM <- head(BRVM_columns_desc, 39)

# Print a message indicating the display of Realtime BRVM Data Columns
cat('Realtime BRVM Data Columns')

# Print the Realtime_BRVM data frame
Realtime_BRVM

Realtime BRVM Data Columns

,Ticker,Sector,Company,Country,Date,Open,High,Low,Close,Volume,Market Cap,Market Percentage
,<chr>,<fct>,<chr>,<fct>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<formttbl>,<dbl>
1,SNTS,PUBLIC SERVICE,SONATEL SENEGAL,SENEGAL,2024-03-20,17850,17885,17860,17885,1834,"1,789,500,000,000",22.54
2,ORAC,PUBLIC SERVICE,ORANGE COTE D'IVOIRE,IVORY COAST,2024-03-20,10800,10800,10750,10750,1563,"1,625,571,226,500",20.47
3,SGBC,FINANCE,SOCIETE GENERALE COTE D'IVOIRE,IVORY COAST,2024-03-20,17650,17750,17650,17700,469,"549,422,202,600",6.92
4,ECOC,FINANCE,ECOBANK COTE D'IVOIRE,IVORY COAST,2024-03-20,6800,6800,6700,6780,1342,"373,793,574,000",4.71
5,ETIT,FINANCE,Ecobank Transnational Incorporated TOGO,TOGO,2024-03-20,18,18,17,18,21960,"325,513,920,996",4.10
6,SIBC,FINANCE,SOCIETE IVOIRIENNE DE BANQUE COTE D'IVOIRE,IVORY COAST,2024-03-20,5790,5790,5600,5780,446,"283,250,000,000",3.57
7,CBIBF,FINANCE,CORIS BANK INTERNATIONAL BURKINA FASO,BURKINA FASO,2024-03-20,8665,8750,8665,8700,62,"280,000,000,000",3.53
8,BOABF,FINANCE,BANK OF AFRICA BURKINA FASO,BURKINA FASO,2024-03-20,7295,7300,7200,7295,678,"160,380,000,000",2.02
9,ORGT,FINANCE,ORAGROUP TOGO,TOGO,2024-03-20,2200,2295,2200,2200,252,"152,713,068,200",1.92


In [36]:
# Retrieve BRVM index data and store it in brvm_sector
brvm_sector <- BRVM_index()

# Rename the column "Indexes" to "Sector" in the brvm_sector dataframe
brvm_sector <- brvm_sector %>%
  rename(Sector = Indexes)

# Print a message indicating that BRVM Indexes are being loaded
print("Loading BRVM Indexes")

# Print the brvm_sector dataframe
brvm_sector

[1] "Loading BRVM Indexes"


Sector,Previous closing,Closing,Change (%),Year to Date Change
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
BRVM-30,107.33,107.19,-0.13,-0.36
BRVM - AGRICULTURE,174.28,173.58,-0.40,2.09
BRVM - OTHER SECTOR,993.28,943.92,-4.97,0.00
BRVM - COMPOSITE,213.80,213.43,-0.17,-0.36
BRVM - DISTRIBUTION,313.06,313.91,0.27,1.12
BRVM - FINANCE,86.82,86.37,-0.52,0.45
BRVM - INDUSTRY,102.91,101.83,-1.05,-0.65
BRVM - PRESTIGE,99.72,99.55,-0.17,1.05
BRVM - PRINCIPAL,104.02,103.77,-0.24,-1.06


In [39]:
# Exporting DataFrame into an Excel file for BRVM Sectors

# Write the DataFrame to an Excel file

write.xlsx(brvm_sector, "/Users/quantumsphere/Desktop/BRVM Project/BRVM-Analyzer/BRVM Excel R files/BRVM_Sectors.xlsx", overwrite = TRUE)

In [40]:
# Convert the Date column to character type
Realtime_BRVM$Date <- as.character(Realtime_BRVM$Date)

# Export the DataFrame to an Excel file
write.xlsx(Realtime_BRVM, "/Users/quantumsphere/Desktop/BRVM Project/BRVM-Analyzer/BRVM Excel R files/Realtime_BRVM.xlsx", overwrite = TRUE)

In [41]:
# Define the file path for the CSV file
file_path <- "/Users/quantumsphere/Desktop/BRVM Project/BRVM-Analyzer/BRVM Tableau R files/BRVM_Historical_TL.csv"

# Check if the file already exists
if (file.exists(file_path)) {
  # If the file exists, remove it
  file.remove(file_path)

  # Display message indicating the existing file is removed
  cat("Existing file removed.\n")
}

# Write the new CSV file
write.csv(BRVM_columns_desc, file = file_path, row.names = FALSE)

# Display message indicating the new CSV file is written
cat("New CSV file written successfully.\n")

Existing file removed.
New CSV file written successfully.


In [42]:
# Define the file path for the CSV file
file_path <- "/Users/quantumsphere/Desktop/BRVM Project/BRVM-Analyzer/BRVM Tableau R files/BRVM_Sectors_TL.csv"

# Check if the file already exists
if (file.exists(file_path)) {
  # If the file exists, remove it
  file.remove(file_path)

  # Display message indicating the existing file is removed
  cat("Existing file removed.\n")
}

# Write the new CSV file
write.csv(brvm_sector, file = file_path, row.names = FALSE)

# Display message indicating the new CSV file is written
cat("New CSV file written successfully.\n")

Existing file removed.
New CSV file written successfully.
